# bs4 lib

A biblioteca **BeautifulSoup** (geralmente importada como `bs4`) é uma ferramenta poderosa em Python para fazer **web scraping**. Ela facilita a extração de dados de documentos HTML e XML analisando-os e fornecendo maneiras fáceis de navegar, pesquisar e modificar a árvore de análise.

Pense no BeautifulSoup como um navegador web que lê o código-fonte de uma página e o organiza de forma que você possa encontrar rapidamente os pedaços de informação que procura, mesmo que o HTML não esteja perfeitamente formatado.

In [1]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


## Como o BeautifulSoup Funciona?

1.  **Parsing:** Você passa um documento HTML ou XML (geralmente como uma string ou um objeto de arquivo) para o construtor `BeautifulSoup`.
2.  **Árvore de Análise:** O BeautifulSoup cria uma estrutura de árvore Python a partir do documento, onde cada elemento HTML/XML, string de texto, comentário, etc., é representado como um **objeto**.
3.  **Navegação e Busca:** Você usa os métodos e propriedades desses objetos para navegar pela árvore e encontrar tags, atributos ou textos específicos.

## Mapeamento de Componentes (Objetos) no BeautifulSoup

O BeautifulSoup lida com diferentes tipos de componentes dentro de um documento, cada um representado por uma classe de objeto:

  - **`BeautifulSoup` Object:** Representa o documento inteiro. É o ponto de entrada para todas as operações.
  - **`Tag` Object:** Representa uma tag HTML ou XML (ex: `<p>`, `<div>`, `<a>`). Tags têm nomes, atributos e podem ter conteúdo (filhos).
  - **`NavigableString` Object:** Representa uma string de texto dentro de uma tag. Não tem atributos ou filhos.
  - **`Comment` Object:** Uma subclasse especial de `NavigableString` que representa um comentário HTML/XML.

## Métodos e Propriedades Principais

Vamos mergulhar nos métodos e propriedades mais usados, com exemplos práticos.

Primeiro, preparemos um documento HTML de exemplo:

In [2]:
from bs4 import BeautifulSoup
from data import html_doc # data.py

soup = BeautifulSoup(html_doc, 'html.parser') # 'html.parser' é o parser padrão e mais comum


### 1. Navegando na Árvore (Propriedades)

Você pode acessar tags como propriedades do objeto `BeautifulSoup` ou de outras `Tag`s.

  - **`soup.tag_name`**: Acessa a *primeira* ocorrência da tag.
  - **`tag.parent`**: Acessa o elemento pai da tag.
  - **`tag.children`**: Um gerador para os filhos diretos da tag.
  - **`tag.contents`**: Uma lista dos filhos diretos da tag.
  - **`tag.descendants`**: Um gerador para todos os elementos descendentes da tag (filhos, netos, etc.).
  - **`tag.next_sibling`, `tag.previous_sibling`**: Próximo/anterior irmão da tag (inclui strings de nova linha se houver).
  - **`tag.next_element`, `tag.previous_element`**: Próximo/anterior elemento na ordem de leitura do documento (inclui strings e comentários).


In [3]:
# Acessando a primeira tag <body>
body_tag = soup.body
print(f"Tag <body>: {body_tag.name}")

# Acessando o pai de <body>
print(f"Pai de <body>: {body_tag.parent.name}")

# Acessando o primeiro <h1>
h1_tag = soup.h1
print(f"Tag <h1>: {h1_tag.get_text()}")

# Acessando filhos de <ul>
ul_tag = soup.ul
print("\nFilhos diretos de <ul>:")
for child in ul_tag.children:
    if child.name: # Checa se é uma tag
        print(f"  - {child.name}")

# Acessando todos os descendentes de <header>
header_tag = soup.header
print("\nDescendentes de <header> (primeiros 5):")
for i, descendant in enumerate(header_tag.descendants):
    if i >= 5: break
    if descendant.name:
        print(f"  - <{descendant.name}>")
    elif descendant.strip(): # Se for texto não vazio
        print(f"  - '{descendant.strip()}'")

# Acessando o irmão próximo de <p class="paragrafo-info">
p_info = soup.find('p', class_='paragrafo-info')
next_sibling = p_info.next_sibling
# next_sibling pode ser uma NavigableString de nova linha, então pegamos o próximo elemento real
while next_sibling and not next_sibling.name:
    next_sibling = next_sibling.next_sibling
if next_sibling:
    print(f"\nPróximo irmão real de 'paragrafo-info': <{next_sibling.name}> - {next_sibling.get_text().strip()}")

Tag <body>: body
Pai de <body>: html
Tag <h1>: Bem-vindo ao Meu Site

Filhos diretos de <ul>:
  - li
  - li
  - li

Descendentes de <header> (primeiros 5):
  - <h1>
  - 'Bem-vindo ao Meu Site'
  - <nav>

Próximo irmão real de 'paragrafo-info': <p> - Este é outro parágrafo.


### 2. Atributos de Tags (Propriedades)

Atributos de tags podem ser acessados como se fossem itens de um dicionário, ou usando `.get()`.

  - `tag['attribute_name']`: Acessa o valor de um atributo.
  - `tag.get('attribute_name')`: Uma forma mais segura, retorna `None` se o atributo não existir.
  - `tag.attrs`: Um dicionário contendo todos os atributos da tag.


In [4]:
link_home = soup.find('a', id='home') # Vamos usar find para pegar um elemento específico
print(f"Href do link 'Home': {link_home['href']}")
print(f"Classes do link 'Home': {link_home['class']}") # Retorna uma lista de classes

# Acessando um atributo que pode não existir com .get()
img_tag = soup.find('img')
print(f"Alt da imagem: {img_tag.get('alt')}")
print(f"Src da imagem: {img_tag.get('src')}")
print(f"Atributo 'data-id' da imagem: {img_tag.get('data-id')}")
print(f"Atributo 'non-existent' da imagem (get): {img_tag.get('non-existent')}") # Retorna None
# print(img_tag['non-existent']) # Isso causaria um KeyError

print("\nTodos os atributos da imagem:")
print(img_tag.attrs)

Href do link 'Home': /
Classes do link 'Home': ['menu-link', 'active']
Alt da imagem: Foto histórica
Src da imagem: historia.jpg
Atributo 'data-id' da imagem: 123
Atributo 'non-existent' da imagem (get): None

Todos os atributos da imagem:
{'src': 'historia.jpg', 'alt': 'Foto histórica', 'data-id': '123'}


### 3. Busca de Elementos (Métodos Essenciais)

Esses são os métodos mais usados para encontrar tags na árvore.

  - **`soup.find(name, attrs, recursive, string, **kwargs)`**: Encontra a *primeira* tag que corresponde aos critérios.

  - **`soup.find_all(name, attrs, recursive, string, limit, **kwargs)`**: Encontra *todas* as tags que correspondem aos critérios. Retorna uma lista de `Tag`s.

      - `name`: Nome da tag (ex: `'div'`, `['p', 'span']`, `True` para qualquer tag).
      - `attrs`: Um dicionário de atributos (ex: `{'class': 'menu-link', 'id': 'home'}`).
      - `recursive` (padrão `True`): Se `False`, busca apenas filhos diretos.
      - `string`: Busca tags por seu conteúdo de texto.
      - `limit`: Limita o número de resultados retornados por `find_all`.
      - `**kwargs`: Pode passar atributos diretamente (ex: `class_='menu-link'`, `id='home'`). Note `class_` para evitar conflito com a palavra-chave `class` do Python.

  - **`soup.select(selector, limit=None)`**: Encontra todas as tags que correspondem a um **seletor CSS**. Retorna uma lista de `Tag`s. É o método mais flexível para buscas complexas.

  - **`soup.select_one(selector)`**: Encontra a *primeira* tag que corresponde a um **seletor CSS**.


In [5]:
# Usando find()
primeiro_paragrafo = soup.find('p')
print(f"\nPrimeiro parágrafo: '{primeiro_paragrafo.get_text(strip=True)}'")

# Usando find_all()
todos_links_menu = soup.find_all('a', class_='menu-link')
print("\nTodos os links de menu:")
for link in todos_links_menu:
    print(f"  - {link.get_text(strip=True)} (Href: {link['href']})")

# Buscando por atributos específicos
primeiro_servico_web = soup.find('div', attrs={'data-tipo': 'web'})
print(f"\nPrimeiro serviço web: '{primeiro_servico_web.h4.get_text(strip=True)}'")

# Buscando por texto
paragrafo_historia = soup.find('p', string="Este é um parágrafo sobre a história da empresa.")
print(f"\nParágrafo com texto específico: '{paragrafo_historia.get_text(strip=True)}'")

# Usando select() com seletores CSS
print("\nElementos selecionados por CSS Selector:")
# Seleciona todos os 'p' dentro de uma 'section' com id 'primeira-secao'
paragrafos_secao = soup.select('section#primeira-secao p')
for p in paragrafos_secao:
    print(f"  - P da seção: '{p.get_text(strip=True)}'")

# Seleciona o <span> dentro de um <div class="servico"> cujo data-tipo é "mobile"
span_servico_mobile = soup.select_one('div.servico[data-tipo="mobile"] span')
print(f"  - Descrição do serviço mobile: '{span_servico_mobile.get_text(strip=True)}'")

# Seleciona o primeiro link com a classe 'menu-link' e 'active'
link_ativo = soup.select_one('a.menu-link.active')
if link_ativo:
    print(f"  - Link ativo: '{link_ativo.get_text(strip=True)}'")


Primeiro parágrafo: 'Este é um parágrafo sobre a história da empresa.'

Todos os links de menu:
  - Home (Href: /)
  - Sobre Nós (Href: /sobre)
  - Contato (Href: /contato)

Primeiro serviço web: 'Desenvolvimento Web'

Parágrafo com texto específico: 'Este é um parágrafo sobre a história da empresa.'

Elementos selecionados por CSS Selector:
  - P da seção: 'Este é um parágrafo sobre a história da empresa.'
  - P da seção: 'Este é outro parágrafo.'
  - Descrição do serviço mobile: 'Aplicativos inovadores para Android e iOS.'
  - Link ativo: 'Home'


### 4. Acessando Conteúdo de Texto

  - `tag.get_text(separator="", strip=False)`: Extrai todo o texto contido em uma tag e seus filhos.
      - `separator`: String para colocar entre o texto de tags aninhadas.
      - `strip`: Se `True`, remove espaços em branco extras do início/fim.
  - `tag.string`: Acessa o texto direto, se a tag tiver apenas um nó de texto filho. Retorna `None` ou o primeiro texto se houver múltiplos filhos.


In [ ]:
h1_text = soup.h1.get_text()
print(f"Texto do H1 (com espaços): '{h1_text}'")

h1_stripped_text = soup.h1.get_text(strip=True)
print(f"Texto do H1 (sem espaços): '{h1_stripped_text}'")

# Texto de um span dentro de um div de serviço
dev_web_span = soup.select_one('div[data-tipo="web"] span')
print(f"Texto do span 'Desenvolvimento Web': '{dev_web_span.string}'")

# Exemplo de tag.string vs. tag.get_text() com filhos múltiplos
nav_tag = soup.nav
# print(nav_tag.string) # Isso retornaria None ou o primeiro texto direto, não o texto combinado

# Isso extrai todo o texto dentro da nav, incluindo de seus filhos (ul, li, a)
print(f"\nTexto completo da navegação (get_text): '{nav_tag.get_text(strip=True, separator=' | ')}'")

### 5. Modificando a Árvore (Métodos)

BeautifulSoup também permite modificar a árvore, o que pode ser útil antes de salvar o HTML de volta ou para testes.

  - `tag.append(child)`: Adiciona um filho no final da tag.
  - `tag.insert(position, child)`: Insere um filho em uma posição específica.
  - `tag.extract()`: Remove a tag do documento e a retorna.
  - `tag.decompose()`: Remove a tag do documento, sem retorná-la.
  - `tag.replace_with(new_tag)`: Substitui a tag por uma nova tag.
  - `tag.wrap(wrapper_tag)`: Envolve a tag com outra tag.
  - `tag.unwrap()`: Remove a tag pai e mantém seus filhos.
  - `tag['attr'] = 'new_value'`: Modifica o valor de um atributo.
  - `del tag['attr']`: Remove um atributo.
  - `tag.string = 'new text'`: Altera o texto direto de uma tag (se ela tiver apenas um filho de texto).


In [6]:
# Modificando o texto de um parágrafo
footer_p = soup.select_one('footer p')
footer_p.string = "© 2025 Exemplo - Nova mensagem de direitos."
print(f"\nParágrafo do rodapé modificado: '{footer_p.get_text(strip=True)}'")

# Adicionando uma nova classe a um link
scroll_top_link = soup.select_one('a.scroll-top')
if scroll_top_link:
    scroll_top_link['class'].append('active-scroll')
    print(f"Classes do link 'Voltar ao topo' após adição: {scroll_top_link['class']}")

# Removendo um atributo
del img_tag['alt']
print(f"Atributos da imagem após remover 'alt': {img_tag.attrs}")

# Criando e adicionando um novo elemento
new_li = soup.new_tag('li')
new_a = soup.new_tag('a', href="/blog")
new_a['class'] = ['menu-link']
new_a.string = "Blog"
new_li.append(new_a)

soup.ul.append(new_li) # Adiciona o novo item ao menu
print("\nHTML do menu após adicionar 'Blog':")
print(soup.ul.prettify()) # prettify() para imprimir formatado

# Removendo um elemento
servico_mobile_div = soup.select_one('div.servico[data-tipo="mobile"]')
if servico_mobile_div:
    servico_mobile_div.extract()
    print("\nDiv 'Desenvolvimento Mobile' removida do HTML.")
    # Para ver o HTML completo após a remoção:
    # print(soup.prettify())


Parágrafo do rodapé modificado: '© 2025 Exemplo - Nova mensagem de direitos.'
Classes do link 'Voltar ao topo' após adição: ['scroll-top', 'active-scroll']
Atributos da imagem após remover 'alt': {'src': 'historia.jpg', 'data-id': '123'}

HTML do menu após adicionar 'Blog':
<ul>
 <li>
  <a class="menu-link active" href="/" id="home">
   Home
  </a>
 </li>
 <li>
  <a class="menu-link" href="/sobre">
   Sobre Nós
  </a>
 </li>
 <li>
  <a class="menu-link" href="/contato">
   Contato
  </a>
 </li>
 <li>
  <a class="menu-link" href="/blog">
   Blog
  </a>
 </li>
</ul>


Div 'Desenvolvimento Mobile' removida do HTML.
